# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="029_citation_query_engine",
    RUN_DESCRIPTION="""
# Objective
Try Citation Query Engine to properly do the citation

# Implementation

## Changelog
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=True,
    RECREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    DEBUG=False,
)

ARGS

2024-08-17 12:10:36.373 | WARNING  | src.run.args:check_recreate_flags:33 - If RECREATE_INDEX is True, then RECREATE_RETRIEVAL_EVAL_DATASET should also be True. Automatically setting RECREATE_RETRIEVAL_EVAL_DATASET to True.
2024-08-17 12:10:36.374 | WARNING  | src.run.args:check_recreate_flags:38 - If RECREATE_INDEX is True, then RECREATE_RESPONSE_EVAL_DATASET should also be True. Automatically setting RECREATE_RESPONSE_EVAL_DATASET to True.


{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "029_citation_query_engine",
  "RUN_DESCRIPTION": "\n# Objective\nTry Citation Query Engine to properly do the citation\n\n# Implementation\n\n## Changelog\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true,
  "RECREATE_INDEX": true
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-17 12:10:36.416 | INFO     | src.run.cfg:init:151 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-17 12:10:40.879 | INFO     | src.run.cfg:init:169 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 029_citation_query_engine...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-17 12:10:41.110 | INFO     | src.run.cfg:init:178 - Notebook-generated artifacts are persisted at data/029_citation_query_engine
2024-08-17 12:10:41.110 | INFO     | src.run.cfg:init:184 - ARGS.RECREATE_INDEX=True -> Overwriting db_collection and storage_context_persist_dp...


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "029_citation_query_engine",
    "RUN_DESCRIPTION": "\n# Objective\nTry Citation Query Engine to properly do the citation\n\n# Implementation\n\n## Changelog\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true,
    "RECREATE_INDEX": true
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/029_citation_query_engine/storage_context",
  "vector_db": "qdrant",
  "db_collection": "review_rec_bot__029_citation_query_engine",
  "db_collection_fp": "data/029_citation_query_engine/qdrant",
  "notebook_cache_dp": "data/029_citation_query_engine",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "openai",
    "llm_model_name": "gpt-4o-mini",
    "embedding_provider": "huggingface",
  

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    # ChromaDB allows only str, int, float dtypes to be indexed as metadata
    # biz_categories=lambda df: df['biz_categories'].str.split(', '),
    biz_categories=lambda df: df['biz_categories'].astype(str),
    biz_attributes=lambda df: df['biz_attributes'].astype(str),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S'),
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-17 12:10:41.294 | INFO     | __main__:<module>:9 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
data.dtypes

review_id            object
user_id              object
business_id          object
review_stars          int64
useful                int64
funny                 int64
cool                  int64
text                 object
date                 object
biz_name             object
biz_address          object
biz_city             object
biz_state            object
biz_postal_code      object
biz_latitude        float64
biz_longitude       float64
biz_stars           float64
biz_review_count      int64
biz_is_open           int64
biz_attributes       object
biz_categories       object
biz_hours            object
dtype: object

In [13]:
input_data = data
if ARGS.TESTING:
    input_data = data[:30]  # 30 examples to avoid bm25 retreiver error...
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-17 12:10:41.350 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [14]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [15]:
from llama_index.core import Document

documents = []
metadata_excluded_cols = ('biz_hours', 'text')
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories", "business_id"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in metadata_excluded_cols}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-17 12:10:41.882 | INFO     | __main__:<module>:21 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [16]:
from llama_index.core.schema import MetadataMode

In [17]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [18]:
llm, embed_model = cfg.setup_llm()

WARNI [sentence_transformers.SentenceTransformer] You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [19]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "openai",
  "llm_model_name": "gpt-4o-mini",
  "embedding_provider": "huggingface",
  "embedding_model_name": "./data/finetune_embedding/finetuned_model",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [20]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [21]:
if cfg.vector_db == 'chromadb':
    from src.run.vector_db import ChromaVectorDB as VectorDB
elif cfg.vector_db == 'qdrant':
    from src.run.vector_db import QdrantVectorDB as VectorDB

In [22]:
vector_db = VectorDB(cfg)
vector_store = vector_db.vector_store
db_collection_count = vector_db.doc_count
logger.info(f"{db_collection_count=}")

2024-08-17 12:10:45.708 | INFO     | src.run.vector_db.qdrant:_setup_db:40 - Deleting existing Qdrant collection review_rec_bot__029_citation_query_engine...
2024-08-17 12:10:45.714 | INFO     | src.run.vector_db.qdrant:_setup_db:45 - Deleting persisted storage context at data/029_citation_query_engine/storage_context...
2024-08-17 12:10:45.715 | INFO     | src.run.vector_db.qdrant:_setup_db:49 - Creating new Qdrant collection review_rec_bot__029_citation_query_engine...
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.
2024-08-17 12:10:46.119 | INFO     | __main__:<module>:4 - db_collection_count=0


# Index Embeddings

In [23]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
# from llama_index.core.node_parser import SentenceSplitter

In [24]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [25]:
t0 = time.perf_counter()

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-17 12:10:46.239 | INFO     | __main__:<module>:12 - Creating new DB index...
2024-08-17 12:10:46.245 | INFO     | __main__:<module>:33 - Running Ingestion Pipeline with num_workers=None...


Parsing nodes:   0%|          | 0/5240 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/18 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/28 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2555/2555 [03:52<00:00, 11.01it/s]
2024-08-17 12:22:46.645 | INFO     | __main__:<module>:43 - Persisting Storage Context to data/029_citation_query_engine/storage_context...


In [26]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-17 12:22:50.031 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 724s
2024-08-17 12:22:50.031 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [27]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'touristy\': False, \'hipster\': True, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'l

# Construct Retriever

In [28]:
# from src.run.retriever.auto_retrieval import get_retriever
from src.run.retriever.retriever import get_retriever

In [29]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = get_retriever(cfg, index)

In [30]:
query = "Nashville, grocery"
vector_retrieval_results = vector_retriever.retrieve(query)
for result in vector_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

**Node ID:** ad57418d-80f9-4081-9fda-b4aa2cd730e7<br>**Similarity:** 0.5700424<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>**Metadata:** {'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

**Node ID:** 55754bf6-80ec-48f0-8f8f-92c4c683625e<br>**Similarity:** 0.56676066<br>**Text:** The food was decent once we got it but honestly, not all that great considering the hassle you have to go through to get it and eat. There are a million great places to eat in Nashville...I'd put this low on the list.<br>**Metadata:** {'review_id': 'zf2I-PCmKYTh2pWZgJ2PZw', 'user_id': 'K1tkMX_b5NAirPCyfyZvyA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 2, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2019-08-19T19:05:21', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

**Node ID:** 802ce50c-42f2-4c5a-b23c-37fc9bc2378e<br>**Similarity:** 0.56634337<br>**Text:** A must stop in Nashville<br>**Metadata:** {'review_id': 'BbJRG7HkGx0S3SuvvpVvIQ', 'user_id': '26nVu2FZTrYt3PEMXLCR0A', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-09-28T04:48:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 1e19d063-6570-485e-a452-2dc1aa3c3085<br>**Similarity:** 0.56415075<br>**Text:** Went back two out of three nights in Nashville.<br>**Metadata:** {'review_id': 'c6X51SW8ip98-tUeCdrBvQ', 'user_id': '6wsCP_ZTebA6xYB2TzhgOQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-05-02T06:30:32', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 86c5bdc1-fd5f-4149-9e00-b758a79e6670<br>**Similarity:** 0.56364906<br>**Text:** Prices are cheap as well! This was a great sweets option in Downtown Nashville.<br>**Metadata:** {'review_id': 'UB_KQiHpvPx8We7bpHb92w', 'user_id': 'hYWwocw-J4KdyGLrUlOYXA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 1, 'funny': 1, 'cool': 1, 'date': '2016-10-16T21:29:55', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

...

**Node ID:** 559c851d-c5e0-409e-93c7-a91719fdf96b<br>**Similarity:** 0.5296338<br>**Text:** In Nashville for a few days, craving some ice cream on a hot and muggy night in June. All homemade ice creams. A lot of choices. Went with the mint white chocolate Oreo and the coffee fudge. Both were amazing.<br>**Metadata:** {'review_id': 'hhrKrr72PxyqfPExpnIhBg', 'user_id': 'HcK_Z02XNy-5Mx1h97WzGA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-06-23T22:37:42', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 7f5301f6-261f-4d45-877a-3f648ac01428<br>**Similarity:** 0.5296031<br>**Text:** Don't even get me started on their in-house waffle cones! I don't even live in Nashville, but when I come back up to these parts I'm definitely stopping here again.<br>**Metadata:** {'review_id': 'ze-CSDGhOtNQjn4JWi1Axw', 'user_id': '_IAkGZyZMKhPpIw08mrZvw', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2014-06-18T11:12:31', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 50994aaf-a0cf-48e0-a664-599f1de7132c<br>**Similarity:** 0.5295278<br>**Text:** I can never say no to ice cream - and when I saw pictures on Yelp of the red velvet ice cream? Nothing could stop me from leaving Nashville without a scoop of it. 

There was a line nearly out the door around 7pm, but we figured it would be worth the wait, especially after having to take an uber all the way back to downtown Nashville. 

There are probably 3-5 tables inside, with 5-10 chairs available to sit at.<br>**Metadata:** {'review_id': 'f_s8UNxtL7kReHvZ7FTGJA', 'user_id': 'O3nhfuz6W4wlmIQgM_Oo2g', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2016-04-06T13:08:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 2dfa310f-2132-4fe7-a854-56ff9145ac09<br>**Similarity:** 0.52908885<br>**Text:** I would love to go back sometime and try out some more of their flavors.

If you are in Nashville, I would highly recommend putting it on your list of places to visit... especially if you are a fan of ice cream.<br>**Metadata:** {'review_id': 'ujq0KAsSvdjs2u1IKASd7A', 'user_id': 'UQ8PA5Rq1D6cnH8hzSDKdA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2012-03-01T03:14:38', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 6f66c08b-9ea1-4eb6-8596-b571f7ce917d<br>**Similarity:** 0.5285912<br>**Text:** It was very good! A classic  when you go to Nashville! It was very busy when we went and unfortunately we had a group of 13.<br>**Metadata:** {'review_id': '3hHtFnNKQFGvRVoBA4mllg', 'user_id': 'RwFB2ewD_yZO546T7q2CGQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 4, 'useful': 1, 'funny': 2, 'cool': 0, 'date': '2013-09-27T00:04:40', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

In [31]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 04119a1f-748c-4963-aa9d-302f33ce1901<br>**Similarity:** 3.2125637531280518<br>**Text:** Convenient grocery store. A bit ghetto, and not because of it's location at all.<br>**Metadata:** {'review_id': '-kdFaer2K56NJCGNadTe5w', 'user_id': 'UYgAC9mvMYc12Ufc5KVUnA', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2010-10-27T15:07:05', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** d65ce092-229b-4960-bdf5-18656c16b027<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** aacc6fff-3622-4894-a01a-a71f92d4661e<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': None, \'Ambience\': None, \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': None, \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': None, \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': None, \'Music\': None, \'NoiseLevel\': None, \'OutdoorSeating\': None, \'RestaurantsAttire\': None, \'RestaurantsDelivery\': None, \'RestaurantsGoodForGroups\': None, \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': None, \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': None, \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': None}', 'biz_categories': 'Food, Grocery'}<br>

**Node ID:** f713fbc6-0b85-48af-ad78-ba70be34e07e<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

...

**Node ID:** a1397e98-ff1c-4706-b2ec-93d4dab90af1<br>**Similarity:** 2.1714556217193604<br>**Text:** Handcrafted ice cream right in the heart of Nashville. The place is always busy but worth the wait.<br>**Metadata:** {'review_id': '073JO4AkBwFz-6VoFSB--A', 'user_id': 'wa1PJGn1XxhIFbGWTeIbOQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-07-07T22:57:17', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

**Node ID:** 5c49c319-de13-45a4-a4b5-17219955248b<br>**Similarity:** 2.1714556217193604<br>**Text:** I had the Nashville Hot Chicken Salad and it was wonderful! Big enough to eat for lunch and dinner!<br>**Metadata:** {'review_id': 'R2AI3QGNuWNP2sHPd40ewg', 'user_id': '1xY7RkCcAzS-KBhZ0ZEE0w', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2019-10-23T18:24:14', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': False}", \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': True, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': True, \'breakfast\': True}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': "{\'dj\': False, \'background_music\': False, \'no_music\': False, \'jukebox\': False, \'live\': False, \'video\': False, \'karaoke\': False}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'True\', \'WiFi\': "\'free\'"}', 'biz_categories': 'Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners'}<br>

**Node ID:** ab64009b-8ad8-4050-85cb-94ce2edb0588<br>**Similarity:** 2.162130832672119<br>**Text:** the cocktails were great & we ordered plenty of hot sake to enjoy our evening. the main reason why I am giving it 3 stars is because the menu is a little pricey (location is downtown nashville off 3rd) & nothing spectacular stood out for it being a new spot in nashville.<br>**Metadata:** {'review_id': 'OI6hq_7xWuK4JuFpkw_nqw', 'user_id': 'lU_H3xHRiWFkVStKX9sd9g', 'business_id': 'tMkwHmWFUEXrC9ZduonpTg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-12-16T01:36:24', 'biz_name': 'The Green Pheasant', 'biz_address': '215 1st Ave S', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1598858, 'biz_longitude': -86.7731974, 'biz_stars': 4.0, 'biz_review_count': 161, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'full_bar\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': False, \'upscale\': False, \'classy\': True, \'casual\': False}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': True, \'street\': False, \'validated\': True, \'lot\': False, \'valet\': True}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': None, \'dinner\': True, \'brunch\': None, \'breakfast\': None}", \'HappyHour\': \'True\', \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': None, \'RestaurantsReservations\': \'True\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'None\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'free\'"}', 'biz_categories': 'Restaurants, Japanese, Seafood'}<br>

**Node ID:** 20458d7e-ea62-45fa-a86f-2f4182f41971<br>**Similarity:** 2.1461713314056396<br>**Text:** Stopped here on a whim for lunch on our way home to Orlando after a weekend at the beach. Delicious sandwiches, lasagna, ziti and salad. Bought some wonderful Italian groceries including fresh mozzarella  and fresh bread to take home. Very friendly place...family business.<br>**Metadata:** {'review_id': 'LC8mOm9RulAdyNlI9a2GNQ', 'user_id': 'UwVqHjSqTRz3FUgr2TPsKg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2015-12-30T20:36:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'True\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'True\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': None, \'DriveThru\': None, \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': None, \'HasTV\': \'True\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'True\', \'RestaurantsGoodForGroups\': \'False\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': \'True\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Food, Delis, Italian, Bakeries, Restaurants'}<br>

**Node ID:** 2b70543f-2d13-41da-bbe9-2442abf2566b<br>**Similarity:** 2.142127275466919<br>**Text:** After visiting Mike's Ice Cream in Nashville, I am now saddened that my favorite spot is across the country. The home made Vanilla instantly reminded me of my youth when my dad used to make ice cream in the summer. If you are in Nashville, I highly recommend Mike's.<br>**Metadata:** {'review_id': 'rtUBWJgwUptv1fvulsGmnQ', 'user_id': 'E1Gfpm8X0JB-x0E7Gsa2qg', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2012-06-01T21:50:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'touristy\': False, \'hipster\': False, \'divey\': False, \'classy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': None, \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': False, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'Caters\': \'False\', \'CoatCheck\': None, \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': \'False\', \'GoodForDancing\': None, \'GoodForKids\': \'True\', \'GoodForMeal\': None, \'HappyHour\': None, \'HasTV\': \'False\', \'Music\': None, \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'None\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': None, \'RestaurantsTakeOut\': \'True\', \'Smoking\': None, \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food'}<br>

In [32]:
# TODO: Reconsider this test.
# Currently it aims to make sure the vector retriever, 
# when given a piece of text extracted from a node, 
# would return the original node.
# But what is the value of it?

# test_query = nodes[3].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

# Check if the original node in top 10:
found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "Can not find the original node in the top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [33]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [34]:
query = "What are some highly reviewed places to enjoy cold brew coffee in Santa Barbara?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee santa barbara
"I recently visited a fantastic café in Santa Barbara that specializes in cold brew coffee. The atmosphere was cozy, and the staff was friendly. I highly recommend checking out The Good Cup; their cold brew is smooth and refreshing!"


**Node ID:** 0d8a1647-144b-4ec4-b1bc-f57c43ab6700<br>**Similarity:** 0.0756192098471423<br>**Text:** Love this place, my go to breakfast when I'm in Santa Barbara...scones are perfect texture, yogurt and granola; as well as, the breakfast sandwich are all high quality ingredients and delicious!<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

Love this place, my go to breakfast when I'm in Santa Barbara...scones are perfect texture, yogurt and granola; as well as, the breakfast sandwich are all high quality ingredients and delicious!




**Node ID:** 18ae6d71-2c74-459f-aa05-087a7439bfb5<br>**Similarity:** 0.06406972467423287<br>**Text:** I had the Santa Barbara bowl and it was soooo good! Great coffee, great atmosphere, would for sure come back!<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

I had the Santa Barbara bowl and it was soooo good! Great coffee, great atmosphere, would for sure come back!




...

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

When visiting Santa Barbara, this place had by far one of the best avocado toasts I have had for brunch in a while! The portion was great and the feta, cucumbers with the egg all tasted divine!




**Node ID:** 7ffba1dc-ca33-4f97-82e7-3024d5873344<br>**Similarity:** 0.02210552027357108<br>**Text:** We got a blueberry lemon scone, a croissant and a morning bun. The blueberry lemon scone was the business! The morning bun was so good - not too sweet. And the croissant was light and flakey. Coffee was good as well. Definitely make it a point to check this place out if you're in SB!<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

We got a blueberry lemon scone, a croissant and a morning bun. The blueberry lemon scone was the business! The morning bun was so good - not too sweet. And the croissant was light and flakey. Coffee was good as well. Definitely make it a point to check this place out if you're in SB!




**Node ID:** d20e94d8-3257-4dfa-80f7-6837c81b33a7<br>**Similarity:** 0.02177964127649662<br>**Text:** First time in Santa Barbara and we came upon this cute spot! Our family has a lot of dietary restrictions and our server was so knowledgable and helpful guiding us through the menu. Highly recommend!<br>

business_id: IDtLPgUrqorrpqSLdfMhZQ
review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries

First time in Santa Barbara and we came upon this cute spot! Our family has a lot of dietary restrictions and our server was so knowledgable and helpful guiding us through the menu. Highly recommend!




## Retrieval Evaluation

### Synthetic

In [35]:
from src.run.eval import RetrievalEvaluator

In [36]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-17 12:22:57.762 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:36 - Creating new retrieval eval dataset at data/029_citation_query_engine/retrieval_synthetic_eval_dataset.json...
2024-08-17 12:22:57.763 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:40 - Sampling 30 nodes for retrieval evaluation...
2024-08-17 12:22:57.775 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:67 - Creating new synthetic retrieval eval dataset...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:19<00:00,  1.52it/s]
2024-08-17 12:23:17.568 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:74 - Persisting synthetic retrieval eval dataset to data/029_citation_query_engine/retrieval_synthetic_eval_dataset.json...


In [37]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'07b4c1ef-bc09-4af4-8576-41e0c8206907': 'What are some restaurants with a lively atmosphere and a diverse menu?',
 '08394bef-043b-4e54-ab93-ab5b16bc8c97': 'What are some reputable shipping companies known for handling packages without damage?',
 '56e2e4f5-e62b-4eda-b3ea-06a43bd466af': 'What are some affordable parking options in Midtown Nashville?',
 '3c281ef7-a7de-48a0-ace7-fb98395203ee': 'What are some businesses or locations that have received poor reviews in the area?',
 '14cf3bcb-e0a9-459c-9f5e-8cbfe4ba9243': 'What are some ice cream shops that offer a cozy atmosphere and a variety of flavors?',
 '111046e9-2396-40ee-9d7e-291d9607e273': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 '5f12ba3b-947e-4584-a95a-a7942f110777': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 'e236fa76-12c9-42fb-9516-21bb958e85b2': 'What are some cafes known for their excellent breakfast sandwiches and coffee?',
 '2fa5a3b9-8a50-

In [38]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
lively atmosphere diverse menu restaurants
"I recently dined at a place called The Vibe, and it was amazing! The atmosphere was buzzing with energy, and the menu had a fantastic variety of dishes from different cuisines. Definitely a spot to check out for a fun night out!"
Generated queries:
reputable shipping companies package handling damage-free
"I recently used SafeShip for my package delivery, and I was impressed by their careful handling. My items arrived in perfect condition, and their customer service was top-notch!"
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking spots. One great option was the public garage on 21st Avenue, which charges a reasonable hourly rate. There are also street parking options available that are budget-friendly, especially in the evenings."
Generated queries:
poor reviews businesses locations area
Generated queries:
ice cream shops cozy atmosphe

In [39]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.7,0.346332,0.014,0.7,0.346332,0.032607


In [40]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some restaurants with a lively atmosp...,[20654c81-1ee1-40a3-a5cc-e4a5457dbae6],"[9c264025-271d-4c33-85a6-7581d1739e8b, a05cd68...",1.0,0.090909,0.02,1.0,0.090909,0.021627
1,What are some reputable shipping companies kno...,[2143af74-008c-4fed-873d-f29cd86cc7eb],"[2143af74-008c-4fed-873d-f29cd86cc7eb, c239d04...",1.0,1.000000,0.02,1.0,1.000000,0.077533
2,What are some affordable parking options in Mi...,[ad57418d-80f9-4081-9fda-b4aa2cd730e7],"[86c5bdc1-fd5f-4149-9e00-b758a79e6670, ad57418...",1.0,0.500000,0.02,1.0,0.500000,0.048918
3,What are some businesses or locations that hav...,[9b95774d-770a-4c54-a74c-52f84cf1764e],"[bcee2628-6cfb-4e7b-b43f-0680ace25477, 303d7f6...",0.0,0.000000,0.00,0.0,0.000000,0.000000
4,What are some ice cream shops that offer a coz...,[3fdf6b4a-b475-4b29-a56b-b1a9fd77b94a],"[646d38f6-796b-4eef-9e00-4d95a966e45f, d69a25c...",0.0,0.000000,0.00,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[3c79349a-bdb3-48cc-8301-f73c64f27864],"[3c79349a-bdb3-48cc-8301-f73c64f27864, bfbe39c...",1.0,1.000000,0.02,1.0,1.000000,0.077533
6,What are some sushi restaurants that offer a l...,[939c4ceb-183b-4d3a-a89b-257228539759],"[c26a8eb5-f262-40d9-a52a-b4c51d5cea70, 939c4ce...",1.0,0.500000,0.02,1.0,0.500000,0.048918
7,What are some cafes known for their excellent ...,[7ed2d00e-b603-4f1f-9291-ca016d2e10e8],"[f2574cd2-a7dd-4731-9993-5fbb4dc56f77, b59d561...",1.0,0.166667,0.02,1.0,0.166667,0.027618
8,What are some alternative credit card companie...,[686b14c9-6e98-4832-aa20-f67b63c588ac],"[d8a6cb6d-fcbe-4936-9837-5725c7c84f16, 7e410a4...",1.0,0.034483,0.02,1.0,0.034483,0.015801
9,What are some restaurants that are known for t...,[1399e2bf-b6ca-4ea8-86e9-e2f700727359],"[aa6805e3-7051-4542-9a71-92a27b081e5b, 750c800...",1.0,0.050000,0.02,1.0,0.050000,0.017652


#### Error Analysis

In [41]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or locations that have received poor reviews in the area?,[9b95774d-770a-4c54-a74c-52f84cf1764e],"[bcee2628-6cfb-4e7b-b43f-0680ace25477, 303d7f6d-a2b2-4da9-b778-cb4011cb0b03, 00c83f53-f9cf-414b-9141-661341c0b89c, 7e04a83b-ddf7-4f9d-b76d-c18c23f3aeb5, ca0454d3-5183-451e-931b-b26137079d34, 0d344a00-e8b0-4c9c-91fd-c5902186c2fc, a5324c99-79c2-4a83-85fa-6c115722f5fb, a0064c16-c3d4-4748-9286-c6ef1a9d3a80, 9d1e0e61-391e-4e60-8f4c-cb56732f55a1, 19ce1f6e-5223-4d3e-b628-d440a2f47705, 86596d35-c57e-4...",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer a cozy atmosphere and a variety of flavors?,[3fdf6b4a-b475-4b29-a56b-b1a9fd77b94a],"[646d38f6-796b-4eef-9e00-4d95a966e45f, d69a25c7-b0c1-411e-8386-972a078a02d3, eaa302f8-f58a-4df9-8d06-33bf8af3d300, b0d363a2-796e-4dc8-aaaf-cb095c010a78, 686b7c24-7ddc-4a82-9f6a-423ee65c7b0c, 9c5fbc30-4355-403b-9cff-527b0b877f40, 566e8baf-e70d-4730-b701-0cc029b97d5b, 287451fc-1999-4fe6-bb38-917d1faa2801, f3d3458b-556f-4204-9841-e915b34a8ab6, 121e6e34-2d64-4c50-906a-6f809773da0c, 17c9b072-56f6-4...",0.0,0.0,0.0,0.0,0.0,0.0
11,What are some ice cream shops that offer a variety of flavors and make their own waffle cones?,[d155e503-3c25-4fc4-8dd5-9f34b5f8b2f5],"[78470d78-538f-44d6-b894-f103f66fbe41, bc644fe7-2415-4f56-9aea-4e8146af26bd, cd9bd465-f59d-4c3f-b1fe-bf9d097696c6, b9a4a621-1f5b-49af-b757-918bcadda36c, 919426ff-c1f6-4933-ba1c-767a884af73b, e7f19b29-a8a7-4060-88a0-adeb8033db8b, 144aedbd-6d89-4ac4-a1cb-f83e28b1505f, 1dcc53ab-f32c-4da9-a80c-de20c085ecd6, d4311654-028c-4ac8-901c-0c0f9d3ee28d, ac1a5be6-6661-47c2-b027-09e1af957c93, 528aa8f6-ebea-4...",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping destinations for clothing and accessories?,[bf6d4d85-65de-4b9d-952f-6f20903bf42a],"[27ca4722-bba6-4db9-bb33-825b4510a641, 90e410f8-39c5-41c9-987f-8f50ae2a18a9, 087fac4a-bc31-4b3d-9bf4-4f440e1d2440, 788488fc-0bb9-4a7a-971c-f8961b4f4301, 6cca2bc9-b4be-4679-9089-48e6d643bcbf, a7d44db7-9ae7-4a99-b859-90d82698bc13, 6e08ffa0-b1c7-467e-9a1c-adc4b6b5c6ea, b5ccd4bb-a434-418c-86d3-2a9026078035, 15c32166-d9fe-4544-a744-3b346c0bd3db, 0d46c234-009f-4e2c-82bb-c1971e2090bd, 46941b49-015f-4...",0.0,0.0,0.0,0.0,0.0,0.0
16,What are some popular attractions or places to visit that offer enjoyable experiences?,[c36aec25-1381-420d-98bb-3c163c14e07d],"[b3b00330-c920-4748-a88b-bf1324478b4e, 8916dd74-54c6-4c30-b701-29e63f8fed68, 9a481eff-99ee-43d0-941d-3c83c3e2e0c6, d2b00898-0eb2-47d0-a5f3-4754f80e722a, 42854261-aee9-4d64-8f32-01e4520b1a95, b7920338-4a2f-4592-876a-39f1531ee521, 30791bcc-99d8-46d2-ae21-addffab98a34, e284b896-0836-449e-a938-0ea6cd134490, 98bd9afd-d8ba-4339-b866-b329cf22de82, de88f9ea-cac2-4401-a652-870e20e3e119, 1e6f2aa4-72ab-4...",0.0,0.0,0.0,0.0,0.0,0.0
17,What are some engaging activities or attractions for children that can create lasting memories?,[33683b0c-741c-47b9-abbf-33c096218cd1],"[575ade68-a45c-460d-9e08-e6214a31081b, 494ec336-ac30-4f79-ad5f-aeeaa5bb15e0, 9b4bd212-676b-4df2-bc05-c7f87f26d168, 2fa3cff3-5622-48e9-819e-63951f802e07, a6a95888-aa03-4284-8160-4bef979ad4b7, e284b896-0836-449e-a938-0ea6cd134490, 84133ad0-f694-4bad-873b-93d00e02deeb, 46f0c601-0c5c-400d-bde8-95c0c045eb1f, df1c39be-a02b-4e02-b011-34cc352e96bb, de88f9ea-cac2-4401-a652-870e20e3e119, 55825156-275b-4...",0.0,0.0,0.0,0.0,0.0,0.0
21,What are some restaurants known for their exceptional customer service and quality food?,[af8f90bd-5eff-4dab-972d-e7b5708a4bda],"[bfe9718d-2134-4ef9-9f85-4893c3285dea, 843a54de-565f-40e7-82b1-0f3e3ccadd8f, 84618b2f-876e-4b93-bdd0-a99a8e6fd65f, 01d867a0-c6a8-404d-bed4-a55ffa4a0c20, 386a67c0-7849-4743-bf1e-e590b61f053c, b1efd408-423f-4f11-9cde-4fb7fe89b5fb, 2eb1986e-853b-4079-844f-fe05079ae384, 6aef1866-26d1-4077-bf81-42422585d6aa, d02130f4-6557-49dd-89d1-1ef302abb45d, 5b7448ea-e831-45bb-9eff-e53e88e407f1, 4bda6077-e877-4...",0.0,0.0,0.0,0.0,0.0,0.0
22,What are some popular tasting rooms or win

In [42]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or locations that have received poor reviews in the area?

- Expected contexts:

business_id: noByYNtDLQAra9ccqxdfDw
review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: Women's Clothing, Accessories, Children's Clothing, Men's Clothing, Adult, Shopping, Fashion

Awful.

- Retrieved contexts:

+ Context #1:

business_id: 7PDi_iyik3jraDAzWwwR4Q
review_stars: 1
biz_name: Chase JP Morgan Bank Credit Card Services
biz_address: 
biz_city: Wilmington
biz_state: DE
biz_categories: Banks & Credit Unions, Financial Services

I will post updates to this situation as they occur. For reference about the dispute, see http://www.yelp.com/user_details_review_search?userid=OfddPlzVNLSwlOTaeBGZKA&q=Metro+PCS or find the Metro PCS review in the L.A. area.

+ Context #2:

business_id: kV_Q1oqis8Qli8dUoGpTyQ
review_stars: 3
biz_name: Ardmore Pizza
biz_address: 10 Rittenhouse P

In [43]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items unique local products
"I recently discovered a fantastic grocery store in Nashville that specializes in curated items and unique local products. They have a great selection of artisanal cheeses and locally sourced produce. It's a must-visit for anyone looking for something special!"


**Node ID:** ad57418d-80f9-4081-9fda-b4aa2cd730e7<br>**Similarity:** 0.060813099943534725<br>**Text:** Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

Interior is gorgeous. Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Pa

**Node ID:** 6334e8b0-cf0b-4695-8b40-49bacf38e82f<br>**Similarity:** 0.05957049486461251<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

business_id: lk9IwjZXqUMqqOhM774DtQ
review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: Coffee & Tea, Restaurants, Wine Bars, Bars, Nightlife, American (Traditional), Event Planning & Services, Food, Caterers, Breakfast & Brunch, Cafes, Diners

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




...

**Node ID:** 466397a4-7ad6-4da6-92df-34d412b19c3d<br>**Similarity:** 0.02285930790603688<br>**Text:** ), but I guess if you have the money it'd be worth the visit. We bought some of their Bourbon Barrel Coffee which was SO not good and very expensive ($17/half pound). I like the concept of the store, but the prices of their products should be adjusted.<br>

business_id: vR9_bhoszcpXbB_APv6Xlw
review_stars: 4
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: Fashion, Shopping, Men's Clothing

), but I guess if you have the money it'd be worth the visit. We bought some of their Bourbon Barrel Coffee which was SO not good and very expensive ($17/half pound). I like the concept of the store, but the prices of their products should be adjusted.




**Node ID:** 15c32166-d9fe-4544-a744-3b346c0bd3db<br>**Similarity:** 0.022704081632653063<br>**Text:** I really like this shop. It has a great selection of clothes, accessories, and all the products are expertly curated by the owner who has impeccable taste. 
Signficant others, this is a great place to find gifts for your beloved man. 
Kind of surprised this shop is in my neighborhood but honored all the same.<br>

business_id: vR9_bhoszcpXbB_APv6Xlw
review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: Fashion, Shopping, Men's Clothing

I really like this shop. It has a great selection of clothes, accessories, and all the products are expertly curated by the owner who has impeccable taste. 
Signficant others, this is a great place to find gifts for your beloved man. 
Kind of surprised this shop is in my neighborhood but honored all the same.




[NodeWithScore(node=TextNode(id_='ad57418d-80f9-4081-9fda-b4aa2cd730e7', embedding=None, metadata={'review_id': 'VZMjK-Moy-fN08mR2m9JUg', 'user_id': 'U5hfpmnqW65_u10C1i06Dw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 4, 'funny': 1, 'cool': 0, 'date': '2017-06-23T12:16:51', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'beer_and_wine\'", \'Ambience\': "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': False, \'trendy\': True, \'upscale\': False, \'classy\': True, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\':

# Response

In [82]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode

# from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine
from llama_index.core.query_engine import CitationQueryEngine

from src.features.synthesize_recommendation.custom_tree_summarize import CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
from src.features.citation.custom_citation_query_engine import CUSTOM_CITATION_QA_TEMPLATE, CUSTOM_CITATION_REFINE_TEMPLATE

In [83]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

query_engine = CitationQueryEngine.from_args(
    index=index,
    retriever=retriever,
    node_postprocessors=node_postprocessors,
    citation_qa_template=CUSTOM_CITATION_QA_TEMPLATE,
    citation_refine_template=CUSTOM_CITATION_REFINE_TEMPLATE
)

# response_synthesizer = get_response_synthesizer(
#     response_mode=ResponseMode.TREE_SUMMARIZE,
#     summary_template=CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
# )
# query_engine = QueryEngine(
#     retriever=retriever,
#     response_synthesizer=response_synthesizer,
#     node_postprocessors=node_postprocessors,
# )

## Test Query Engine

In [84]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [85]:
query = "What are some gastropubs in Indianapolis that are known for their cocktails?"
# query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
gastropubs cocktails indianapolis
"I visited a fantastic gastropub in Indianapolis that had an impressive cocktail menu. The mixologists crafted unique drinks, and the atmosphere was perfect for a night out."


**`Final Response:`** Based on your query about gastropubs in Indianapolis that are known for their cocktails, here are several recommendations ranked by relevance:

### 1. Village Tap Room
- **Address:** 838 Broad Ripple Ave, Indianapolis, IN
- **Categories:** Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife
- **Summary:** Village Tap Room has received numerous positive reviews highlighting its excellent craft cocktails and local beer selection. The head bartender is noted for his expertise in classic cocktails, which adds a unique touch to the drink menu. The food is also well-regarded, making it a great spot for both drinks and dining. Reviewers appreciate the friendly staff and the overall atmosphere, which is a significant improvement over its predecessor, Chumley's [2][4][5][9].

### 2. The Libertine Liquor Bar
- **Address:** 38 E Washington St, Indianapolis, IN
- **Categories:** Gastropubs, Cocktail Bars, Bars, Restaurants
- **Summary:** Known for its innovative cocktail menu and a focus on local ingredients, The Libertine offers a vibrant atmosphere perfect for cocktail enthusiasts. The bar features a rotating selection of seasonal cocktails and a diverse wine list. The food menu complements the drinks well, making it a popular choice for a night out. While specific reviews were not provided, it is well-regarded in the local community for its cocktail offerings.

### 3. The Eagle
- **Address:** 310 Massachusetts Ave, Indianapolis, IN
- **Categories:** Gastropubs, Cocktail Bars, Bars, Restaurants
- **Summary:** The Eagle is celebrated for its fried chicken and Southern-inspired menu, but it also boasts a solid cocktail program. The bar offers a variety of classic and signature cocktails, making it a great place to enjoy a drink in a casual setting. The lively atmosphere and friendly service enhance the experience, although it may not be as cocktail-focused as Village Tap Room.

### Comparison of Options:
- **Village Tap Room** stands out as the top recommendation due to its strong emphasis on craft cocktails and local beers, along with a well-reviewed food menu. The expertise of the bartenders and the overall ambiance make it a prime choice for cocktail lovers.
- **The Libertine Liquor Bar** is a close second, known for its innovative cocktails and local ingredients, appealing to those looking for a unique drinking experience. However, it may not have the same level of food offerings as Village Tap Room.
- **The Eagle** is a solid option for those who enjoy a more casual dining experience with good cocktails, but it may not be as focused on cocktails as the other two options.

In summary, if your primary interest is in cocktails, **Village Tap Room** is the best choice, followed by **The Libertine Liquor Bar** for its creativity and seasonal offerings. **The Eagle** is a good option if you're also looking for a hearty meal alongside your drinks.

Sources:
- [2]: "Much improved over Chumley's. Nice selection of local craft beer. Craft cocktails. Food was very good."
- [4]: "This is NOT Chumleys. Excellent local craft beer selection. Hand crafted cocktails. Great food. Friendly staff. Check this place out!!"
- [5]: "Jeremy, the bar tender, is fantastic. The cocktail and beer menu are tops. Food is also spot on."
- [9]: "I haven't seen Bell's Hopslam anywhere else?? The pretzel bites are great... I'll definitely be a regular."

---

**`Source Node 1/10`**

**Node ID:** 6ed778a9-c8ea-49f7-b3ed-ddac592bbccb<br>**Similarity:** 1.0615234375<br>**Text:** Source 1:
Def not as good as Chumley's. Probably not coming back.<br>**Metadata:** {'review_id': 'KepGokf4WSchz_spuA6tTw', 'user_id': 'BqY1Do0kyrBn00NuBVW9Sw', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 1, 'useful': 1, 'funny': 1, 'cool': 0, 'date': '2016-01-03T01:35:06', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': True, \'thursday\': True, \'sunday\': False, \'saturday\': True}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'False\', \'BusinessParking\': None, \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': False, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'False\', \'HasTV\': \'False\', \'Music\': "{\'jukebox\': True}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'False\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'False\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife'}<br>

...

**Node ID:** 2114921b-e4d4-4e61-b021-934bf09625c3<br>**Similarity:** -0.298095703125<br>**Text:** Source 9:
I don't remember the names of the other two servers, but they were very good.  24 beers on tap is plenty, especially when you have great diversity...which they have!  I haven't seen Bell's Hopslam anywhere else??  The pretzel bites are great (think Hop Cat..they seem exactly the same).  Wings and Brisket sandwich were good bar fare.  My buddy enjoyed the puffy taco.  I'll definitely be a regular.<br>**Metadata:** {'review_id': 's1iVrwUgSjub253_L-o-6g', 'user_id': '0y_xpwQ0TZVTIPh3hes4Pw', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2016-03-05T02:24:36', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': True, \'thursday\': True, \'sunday\': False, \'saturday\': True}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'False\', \'BusinessParking\': None, \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': False, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'False\', \'HasTV\': \'False\', \'Music\': "{\'jukebox\': True}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'False\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'False\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife'}<br>

---

**`Source Node 10/10`**

**Node ID:** 3b28a32c-8c82-456f-a813-3c4c89ff596a<br>**Similarity:** -0.402587890625<br>**Text:** Source 10:
I love what this bar is doing. I enjoy the yoga on Mondays and the different offerings that are available during the week. The cocktails are on point and I am happy that the beers are still very diverse but less abundant than when this was chumleys. It makes a decision easier to make.<br>**Metadata:** {'review_id': '-mGS871hiIFrxDmOSACRMA', 'user_id': 'NITZViTUDL5K6nruYrrHUg', 'business_id': 'L_TT0BFmFwORAMaA83K54A', 'review_stars': 4, 'useful': 1, 'funny': 0, 'cool': 2, 'date': '2016-03-28T21:47:24', 'biz_name': 'Village Tap Room', 'biz_address': '838 Broad Ripple Ave', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46220', 'biz_latitude': 39.8699114, 'biz_longitude': -86.1435773, 'biz_stars': 2.5, 'biz_review_count': 23, 'biz_is_open': 0, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': True, \'thursday\': True, \'sunday\': False, \'saturday\': True}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': \'False\', \'BusinessAcceptsCreditCards\': \'False\', \'BusinessParking\': None, \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'False\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': False, \'dinner\': False, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'False\', \'HasTV\': \'False\', \'Music\': "{\'jukebox\': True}", \'NoiseLevel\': "u\'average\'", \'OutdoorSeating\': \'False\', \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'False\', \'RestaurantsTakeOut\': \'False\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': \'False\', \'WiFi\': "u\'no\'"}', 'biz_categories': 'Gastropubs, Cocktail Bars, Beer Bar, Bars, Restaurants, Nightlife'}<br>

{'6ed778a9-c8ea-49f7-b3ed-ddac592bbccb': {'review_id': 'KepGokf4WSchz_spuA6tTw',
  'user_id': 'BqY1Do0kyrBn00NuBVW9Sw',
  'business_id': 'L_TT0BFmFwORAMaA83K54A',
  'review_stars': 1,
  'useful': 1,
  'funny': 1,
  'cool': 0,
  'date': '2016-01-03T01:35:06',
  'biz_name': 'Village Tap Room',
  'biz_address': '838 Broad Ripple Ave',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46220',
  'biz_latitude': 39.8699114,
  'biz_longitude': -86.1435773,
  'biz_stars': 2.5,
  'biz_review_count': 23,
  'biz_is_open': 0,
  'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "u\'none\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'BYOB\': None, \'BestNights\': "{\'monday\': False, \'tuesday\': False, \'friday\': False, \'wednesday\': True, \'thursday\': True, \'sunday\': False, \'saturday\': True}", \'BikeParkin

## Response Evaluation

In [48]:
from src.run.eval import ResponseEvaluator

In [49]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [50]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-17 12:24:04.147 | INFO     | src.run.eval.response:generate_synthetic_dataset:40 - Sampling 30 documents for response evaluation...
2024-08-17 12:24:04.155 | INFO     | src.run.eval.response:generate_synthetic_dataset:60 - Creating new response eval dataset at data/029_citation_query_engine/response_synthetic_eval_dataset.json...
2024-08-17 12:24:04.156 | INFO     | src.run.eval.response:generate_synthetic_dataset:63 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]
2024-08-17 12:24:53.414 | INFO     | src.run.eval.response:generate_synthetic_dataset:86 - Persisting synthetic response eval dataset at data/029_citation_query_engine/response_synthetic_eval_dataset.json...


In [ ]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

Generated queries:
Japanese restaurants Nashville highly-rated variety dishes


In [ ]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

In [ ]:
response_synthetic_mean_scores_df

In [ ]:
response_synthetic_deep_eval_df

#### Error Analysis

In [ ]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [ ]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score', 'context_relevancy_score'])

In [ ]:
CONTEXT_RELEVANCE_SCORE_THRESHOLD = 0.5

response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['context_relevancy_score'].le(CONTEXT_RELEVANCE_SCORE_THRESHOLD)]
    .sort_values(['context_relevancy_score', 'relevancy_score', 'correctness_score', 'faithfulness_score'])
)

with pd.option_context('display.max_colwidth', 400):
    display(response_synthetic_eval_irrelevance_df)

In [ ]:
for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

In [77]:
# query = "- What are some pizza places known for their quality and flavor in White House, TN?"
query = "What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a sports bar in Indianapolis that had an amazing atmosphere and great food. The menu was diverse, and the service was top-notch, making it a fantastic alternative to Twin Peaks."


**`Final Response:`** Based on your query about sports bars in Indianapolis that offer a better dining experience than Twin Peaks, here are several recommendations ranked by relevance:

### 1. **The Rathskeller**
- **Address:** 401 E Michigan St, Indianapolis, IN
- **Categories:** German Restaurant, Beer Garden, Sports Bar
- **Summary:** The Rathskeller is known for its authentic German cuisine and extensive beer selection, including local craft beers. The atmosphere is vibrant, with outdoor seating and live music events. It offers a more upscale dining experience compared to typical sports bars, making it suitable for families and groups looking for a unique dining experience while enjoying sports.

### 2. **Buffalo Wild Wings**
- **Address:** Multiple locations in Indianapolis
- **Categories:** Sports Bar, American (Traditional), Bar
- **Summary:** Buffalo Wild Wings is a well-known chain that specializes in wings and offers a wide variety of sauces. The atmosphere is lively, with numerous TVs for sports viewing. While it may not be as upscale as The Rathskeller, it provides a solid dining experience with a focus on casual dining and sports, making it a popular choice for sports fans.

### 3. **The Union Jack Pub**
- **Address:** 96th St & College Ave, Indianapolis, IN
- **Categories:** British Pub, Sports Bar, Restaurant
- **Summary:** The Union Jack Pub offers a cozy atmosphere with a British twist. The menu features traditional pub fare, including fish and chips, burgers, and a selection of beers. The pub is known for its friendly service and is a great place to watch sports in a more relaxed setting compared to Twin Peaks.

### 4. **The Score**
- **Address:** 1001 W 86th St, Indianapolis, IN
- **Categories:** Sports Bar, American (New), Nightlife
- **Summary:** The Score is a modern sports bar that offers a diverse menu, including gourmet burgers and craft cocktails. The ambiance is upscale yet casual, making it a great spot for both dining and watching games. It provides a more refined experience than Twin Peaks, with a focus on quality food and service.

### 5. **The Tap**
- **Address:** 306 N Delaware St, Indianapolis, IN
- **Categories:** Sports Bar, American (New), Brewery
- **Summary:** The Tap is a local favorite known for its craft beer selection and delicious pub food. The atmosphere is laid-back, with plenty of TVs for sports viewing. While it may not be as themed as Twin Peaks, it offers a better dining experience with a focus on quality ingredients and local brews.

### Comparison of Options:
- **Dining Experience:** The Rathskeller stands out for its unique German cuisine and beer garden atmosphere, making it the top choice for a better dining experience. The Score and The Tap also offer quality food but with a more casual vibe.
- **Family-Friendly:** The Rathskeller and The Union Jack Pub are more family-friendly compared to Twin Peaks, which is often seen as more of a "guys' bar."
- **Atmosphere:** If you're looking for a lively atmosphere with live music, The Rathskeller is ideal. For a more relaxed pub experience, The Union Jack Pub is a great option.

In summary, while Twin Peaks offers a specific themed experience, these alternatives provide a broader range of dining options and atmospheres that may better suit your preferences for a sports bar in Indianapolis.

Sources:
- [1]: "The Rathskeller is known for its authentic German cuisine and extensive beer selection."
- [2]: "Buffalo Wild Wings is a well-known chain that specializes in wings and offers a wide variety of sauces."
- [3]: "The Union Jack Pub offers a cozy atmosphere with a British twist."
- [4]: "The Score is a modern sports bar that offers a diverse menu, including gourmet burgers and craft cocktails."
- [5]: "The Tap is a local favorite known for its craft beer selection and delicious pub food."

---

**`Source Node 1/11`**

**Node ID:** 91e92f89-843b-41eb-891c-bf8676e98cc2<br>**Similarity:** 0.86279296875<br>**Text:** Source 1:
Twin Peaks is your typical guys bar, way above Hooters, and on par with tilted kilt.  The thing that puts it over the top is the ICE cold beer served at 29 degrees.<br>**Metadata:** {'review_id': 'KsmnyFmghtOx_IbOG9RINg', 'user_id': 'jB-7q5FxZKho-wlRAzS2lA', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 0, 'date': '2017-10-25T03:00:01', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

...

**Node ID:** e127d1cf-e6f6-4bf5-81ea-eb07871f596b<br>**Similarity:** -0.33203125<br>**Text:** Source 10:
I'm a big sports bar guy and a sports bar food guy and don't expect a sports bar to be 5 star dining. Let me answer a question that people ask me when I said I ate there. "Is it okay to bring your wife and kids there to watch a game?" The servers do wear low cut shirts and shorts, but nothing that was any different than other themed sports bars. When I was there, there were groups of women together and also couples there.<br>**Metadata:** {'review_id': 'grTir4NbamCVuxnzfa8sqw', 'user_id': 'sZh1Pp958CUE5Tv3y5xtAQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 4, 'useful': 3, 'funny': 1, 'cool': 2, 'date': '2015-11-04T16:52:57', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

---

**`Source Node 11/11`**

**Node ID:** 8b545598-b291-4d53-908c-9852f7b780d2<br>**Similarity:** -0.381591796875<br>**Text:** Source 11:
It's my cuz per   Bachelor party.   The  Waitress at twin peaks (Tyler Walker) is the best and she is a very good waitress. Plus Tyler works hard at all  her tables. Just want the thanks Twin  peaks and Tyler Walker for a very good time.<br>**Metadata:** {'review_id': '8j70_2c-O_L2lWa0nz7KDQ', 'user_id': 'nmqDutrkq9tw_ZrOlLr0FQ', 'business_id': 'seKihQKpGGnCeLuELRQPSQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-09-09T02:37:35', 'biz_name': 'Twin Peaks', 'biz_address': '6880 E 82nd St', 'biz_city': 'Indianapolis', 'biz_state': 'IN', 'biz_postal_code': '46250', 'biz_latitude': 39.9062952, 'biz_longitude': -86.0474634, 'biz_stars': 3.5, 'biz_review_count': 257, 'biz_is_open': 1, 'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeParking\': \'False\', \'BusinessAcceptsBitcoin\': None, \'BusinessAcceptsCreditCards\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': None, \'Caters\': \'False\', \'CoatCheck\': \'False\', \'Corkage\': None, \'DogsAllowed\': \'False\', \'DriveThru\': None, \'GoodForDancing\': \'False\', \'GoodForKids\': \'True\', \'GoodForMeal\': "{\'dessert\': None, \'latenight\': None, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'HappyHour\': \'True\', \'HasTV\': \'True\', \'Music\': "{\'dj\': False}", \'NoiseLevel\': "\'average\'", \'OutdoorSeating\': \'True\', \'RestaurantsAttire\': "\'casual\'", \'RestaurantsDelivery\': \'False\', \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'RestaurantsReservations\': \'False\', \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'Smoking\': "u\'no\'", \'WheelchairAccessible\': None, \'WiFi\': "\'free\'"}', 'biz_categories': 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants'}<br>

{'91e92f89-843b-41eb-891c-bf8676e98cc2': {'review_id': 'KsmnyFmghtOx_IbOG9RINg',
  'user_id': 'jB-7q5FxZKho-wlRAzS2lA',
  'business_id': 'seKihQKpGGnCeLuELRQPSQ',
  'review_stars': 5,
  'useful': 1,
  'funny': 0,
  'cool': 0,
  'date': '2017-10-25T03:00:01',
  'biz_name': 'Twin Peaks',
  'biz_address': '6880 E 82nd St',
  'biz_city': 'Indianapolis',
  'biz_state': 'IN',
  'biz_postal_code': '46250',
  'biz_latitude': 39.9062952,
  'biz_longitude': -86.0474634,
  'biz_stars': 3.5,
  'biz_review_count': 257,
  'biz_is_open': 1,
  'biz_attributes': '{\'AcceptsInsurance\': None, \'Alcohol\': "\'full_bar\'", \'Ambience\': "{u\'divey\': False, u\'hipster\': None, u\'casual\': True, u\'touristy\': None, u\'trendy\': None, u\'intimate\': False, u\'romantic\': False, u\'classy\': False, u\'upscale\': False}", \'BYOB\': None, \'BestNights\': "{u\'monday\': False, u\'tuesday\': True, u\'wednesday\': False, u\'thursday\': False, u\'friday\': True, u\'saturday\': True, u\'sunday\': False}", \'BikeP

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [ ]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

In [ ]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

In [ ]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

In [ ]:
response_curated_mean_scores_df

In [ ]:
response_curated_deep_eval_df

#### Answers for target questions

In [ ]:
from IPython.display import Markdown

In [ ]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [76]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are some places to enjoy cold brew coffee?


Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee, and I was blown away by their unique flavors and smooth texture. It's a perfect spot for coffee lovers!"


**`Final Response:`** Based on your query about places to enjoy cold brew coffee, here are several recommendations ranked by relevance:

### 1. Helena Avenue Bakery
- **Address:** 131 Anacapa St, Ste C, Santa Barbara, CA
- **Categories:** Food, Restaurants, Salad, Coffee & Tea, Breakfast & Brunch, Sandwiches, Bakeries
- **Summary:** Helena Avenue Bakery is highly regarded for its delicious cold brew coffee, which has received multiple positive mentions in reviews. The bakery offers a cozy atmosphere and a variety of food options, including breakfast items and sandwiches. Reviewers have praised the quality of their cold brew, stating it is "so good" and complements their food offerings well. The ambiance is described as chill and inviting, making it a great spot to relax with a drink [1][3][4][5].

### 2. Mike's Ice Cream
- **Address:** 129 2nd Ave N, Nashville, TN
- **Categories:** Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food
- **Summary:** While primarily known for its diverse ice cream flavors, Mike's Ice Cream also serves a variety of coffee options, including cold brew. The establishment is noted for its friendly staff and a cozy environment. Reviewers have highlighted the quality of their coffee, making it a solid choice for those looking to enjoy a cold brew in a more casual setting [6][7][8].

### 3. Roast Coffeehouse and Wine Bar
- **Address:** 10359 104 Street NW, Edmonton, AB
- **Categories:** Coffee & Tea, Food, Cafes, Bars, Wine Bars, Restaurants, Nightlife
- **Summary:** Roast Coffeehouse and Wine Bar offers a unique experience with its combination of coffee and wine. While it has received mixed reviews regarding the quality of its coffee, it does serve cold brew options. The ambiance is described as open and industrial, which may appeal to those looking for a trendy spot. However, some reviewers have expressed disappointment in the coffee quality, so it may not be the best choice if you're seeking top-notch cold brew [9][10].

### Comparison of Options:
- **Helena Avenue Bakery** stands out as the top choice due to its consistent praise for cold brew quality and a welcoming atmosphere, making it ideal for both casual visits and breakfast outings.
- **Mike's Ice Cream** is a great alternative if you're in the mood for a sweet treat alongside your cold brew, offering a fun and relaxed vibe.
- **Roast Coffeehouse and Wine Bar** may appeal to those looking for a unique combination of coffee and wine, but it may not satisfy coffee aficionados seeking high-quality cold brew.

In summary, if your primary goal is to enjoy a high-quality cold brew coffee, **Helena Avenue Bakery** is the best option. If you're looking for a more casual experience with the option of ice cream, consider **Mike's Ice Cream**. For a more eclectic atmosphere with a mix of coffee and wine, **Roast Coffeehouse and Wine Bar** could be worth a visit, but be cautious about the coffee quality.

Sources:
- [1]: "Cold brew was sooo good as were the sandwiches we got. Will most definitely be back."
- [3]: "The cold brew coffee was good. Lots of good choices in the grab and go section."
- [6]: "The staff were friendly and willing to go the extra mile to ensure that we were enjoying our coffee."
- [9]: "Come here for an interesting flavored latte, but don't come if you demand high quality coffee."



### Question: What are some sports bars in Indianapolis that offer a better dining experience than Twin Peaks?


Generated queries:
sports bars dining experience indianapolis better than Twin Peaks
"I visited a great sports bar in Indianapolis called The District Tap. The atmosphere was lively, and they had an extensive menu with delicious food options that surpassed my experience at Twin Peaks."


**`Final Response:`** Based on your query about sports bars in Indianapolis that offer a better dining experience than Twin Peaks, here are several recommendations ranked by relevance:

### 1. **The Rathskeller**
- **Address:** 401 E Michigan St, Indianapolis, IN
- **Categories:** German Restaurant, Beer Garden, Sports Bar
- **Summary:** The Rathskeller is known for its authentic German cuisine and extensive beer selection, including local craft beers. The atmosphere is vibrant, with outdoor seating available in a beautiful beer garden. It offers a more upscale dining experience compared to typical sports bars, making it suitable for families and groups looking for a unique dining experience while enjoying sports.

### 2. **Buffalo Wild Wings**
- **Address:** Multiple locations in Indianapolis
- **Categories:** Sports Bar, American (Traditional), Bar
- **Summary:** Buffalo Wild Wings is a well-known chain that offers a lively sports bar atmosphere with a diverse menu featuring wings, burgers, and a variety of beers. The service is generally reliable, and the environment is family-friendly. While it may not be as upscale as The Rathskeller, it provides a solid dining experience with plenty of screens for sports viewing.

### 3. **The Union Jack Pub**
- **Address:** 96th St & College Ave, Indianapolis, IN
- **Categories:** British Pub, Sports Bar, Restaurant
- **Summary:** The Union Jack Pub offers a cozy British pub atmosphere with a menu that includes traditional British fare alongside American favorites. The pub is known for its friendly service and a good selection of beers. It provides a more relaxed dining experience compared to Twin Peaks, making it a great spot for enjoying sports in a comfortable setting.

### 4. **The Score**
- **Address:** 1001 W 86th St, Indianapolis, IN
- **Categories:** Sports Bar, American (New), Nightlife
- **Summary:** The Score is a modern sports bar that features a diverse menu, including gourmet burgers and unique appetizers. The ambiance is upscale yet casual, with plenty of TVs for sports viewing. It offers a better dining experience than many traditional sports bars, focusing on quality food and a welcoming atmosphere.

### 5. **The Huddle**
- **Address:** 1001 E 86th St, Indianapolis, IN
- **Categories:** Sports Bar, American (Traditional), Bar
- **Summary:** The Huddle is a local favorite known for its friendly service and a menu that includes classic sports bar fare. While it may not be as upscale as some of the other options, it offers a comfortable environment for watching games and enjoying a meal with friends or family.

### Comparison of Options:
- **Dining Experience:** The Rathskeller stands out for its unique German cuisine and beer garden, making it the top choice for a better dining experience. Buffalo Wild Wings and The Union Jack Pub offer solid dining experiences but are more casual.
- **Atmosphere:** The Score provides a modern and upscale atmosphere, while The Huddle offers a more traditional sports bar vibe.
- **Family-Friendly:** All options are family-friendly, but The Rathskeller and The Union Jack Pub may be more appealing for families looking for a sit-down meal.

In summary, if you're looking for a sports bar with a better dining experience than Twin Peaks, The Rathskeller is the top recommendation, followed by Buffalo Wild Wings and The Union Jack Pub for their unique offerings and welcoming atmospheres.

Sources:
- [1]: "The Rathskeller is known for its authentic German cuisine and extensive beer selection."
- [2]: "Buffalo Wild Wings is a well-known chain that offers a lively sports bar atmosphere."
- [3]: "The Union Jack Pub offers a cozy British pub atmosphere with a menu that includes traditional British fare."
- [4]: "The Score is a modern sports bar that features a diverse menu, including gourmet burgers."
- [5]: "The Huddle is a local favorite known for its friendly service and a menu that includes classic sports bar fare."

### Error Analysis

In [ ]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [ ]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

In [ ]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [ ]:
from src.run.utils import parse_collect_log, flatten_dict

In [ ]:
collect_info = parse_collect_log(collect_fp)

In [ ]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

# Clean up

In [ ]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

In [75]:
1

1

# Chat Bot Quick Test

In [85]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from src.svc.availability.availability_check import ReservationService

In [86]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="reco_review",
        description=(
            "useful for when you want to find places to visit"
            " based on end-user reviews"
        ),
    ),
)

opening_hours_db = data.set_index('business_id')['biz_hours'].dropna().to_dict()
biz_name_id_mapper = data.set_index('biz_name')['business_id'].dropna().to_dict()
rez_tool = ReservationService(opening_hours_db, biz_name_id_mapper)

tools = [query_engine_tool] + rez_tool.to_tool_list()

In [87]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [88]:
response = agent.chat("hi, i am quy")
print(str(response))

response = agent.chat(
    "Where to find some good cold brew coffee?"
)
print(str(response))

response = agent.chat(
    "where is it?"
)
print(str(response))

response = agent.chat(
    "what is the best time to visit?"
)
print(str(response))

Added user message to memory: hi, i am quy
Hello Quy! How can I assist you today?
Added user message to memory: Where to find some good cold brew coffee?
=== Calling Function ===
Calling function: reco_review with args: {"input":"cold brew coffee"}
Generated queries:
cold brew coffee
"I recently tried a new café that specializes in cold brew coffee, and I was amazed by the smooth flavor and rich aroma. They also offer various blends and seasonal flavors that are perfect for any coffee lover."
Got output: 
Based on your interest in cold brew coffee, here are several recommendations ranked by relevance:

### 1. **Helena Avenue Bakery**
   - **Location:** 131 Anacapa St, Ste C, Santa Barbara, CA
   - **Rating:** 5 stars
   - **Strengths:** Helena Avenue Bakery is highly praised for its cold brew coffee, which has received multiple mentions in reviews for being exceptionally good. The bakery also offers a variety of delicious breakfast options, including sandwiches and pastries, making it 

In [89]:
response = agent.chat(
    "Can I book for 5 pax on August 15 at 8 AM?"
)
print(str(response))

Added user message to memory: Can I book for 5 pax on August 15 at 8 AM?
=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Helena Avenue Bakery"}
Got output: IDtLPgUrqorrpqSLdfMhZQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Caviar & Bananas"}
Got output: lk9IwjZXqUMqqOhM774DtQ

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Wawa"}
Got output: ppFCk9aQkM338Rgwpl2F5A

=== Calling Function ===
Calling function: get_business_id with args: {"biz_name": "Barnes & Noble Booksellers"}
Got output: qhDdDeI3K4jy2KyzwFN53w

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "IDtLPgUrqorrpqSLdfMhZQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: True

=== Calling Function ===
Calling function: check_availability with args: {"business_id": "lk9IwjZXqUMqqOhM774DtQ", "date_time": "2023-08-15T08:00:00", "pax": 5}
Got output: False

=== Callin

In [77]:
agent.reset()

# Chatbot UI

# Archive